I have read from this [link](https://medium.com/donato-story/exploring-few-shot-prompts-with-langchain-852f27ea4e1d)

In [ ]:
import logfire
logfire.configure(send_to_logfire="if-token-present")
logfire.instrument_httpx()

# 1. Simple example

In [2]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    { "input": "2 + 2", "output": "4" },
    { "input": "2 + 3", "output": "5" }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

print(example_prompt.format(input=examples[0]['input'], output=examples[0]['output']))
print(example_prompt.format(input=examples[1]['input'], output=examples[1]['output']))

Human: 2 + 2
AI: 4
Human: 2 + 3
AI: 5


In [3]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.format())

Human: 2 + 2
AI: 4
Human: 2 + 3
AI: 5


In [4]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are super duper wizard of math"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

print(final_prompt.format(input="What man is a man, who doesn’t make the world a better place?"))

System: you are super duper wizard of math
Human: 2 + 2
AI: 4
Human: 2 + 3
AI: 5
Human: What man is a man, who doesn’t make the world a better place?


## 2. Dynamic Examples

In [22]:
examples = [
    { "input": "1 + 1", "output": "2" },
    { "input": "2 + 3", "output": "5" },
    { "input": "6 x 5", "output": "30" },
    { "input": "What did the stone say to the moon?", "output": "He is singing a song like he always need her" },
    { "input": "What a girl want?", "output": "There is no one size fit all" },
    { "input": "How to get a lot of gold?", "output": "type .. iseedeadpeople and press Enter" },
]

In [23]:
examples

[{'input': '1 + 1', 'output': '2'},
 {'input': '2 + 3', 'output': '5'},
 {'input': '6 x 5', 'output': '30'},
 {'input': 'What did the stone say to the moon?',
  'output': 'He is singing a song like he always need her'},
 {'input': 'What a girl want?', 'output': 'There is no one size fit all'},
 {'input': 'How to get a lot of gold?',
  'output': 'type .. iseedeadpeople and press Enter'}]

In [24]:
to_vectorize = [" ".join(example['input']) for example in examples]
to_vectorize

['1   +   1',
 '2   +   3',
 '6   x   5',
 'W h a t   d i d   t h e   s t o n e   s a y   t o   t h e   m o o n ?',
 'W h a t   a   g i r l   w a n t ?',
 'H o w   t o   g e t   a   l o t   o f   g o l d ?']

In [25]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

embeddings_model = OllamaEmbeddings(
    model="bge-m3:latest",
    temperature=0,
)

# ensure no data
vector_store = Chroma(embedding_function=embeddings_model)
vector_store.reset_collection()
vector_store.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [26]:
vector_store.add_texts(to_vectorize, examples)
vector_store.get()

14:03:20.997 POST 127.0.0.1/api/embed


{'ids': ['478870b7-a70c-40f1-bb4a-e3cafaed92aa',
  '65622760-a30e-4e87-a4e6-2cf24886ecbb',
  'c08b682b-3aaa-4d37-bf13-5225b86711bf',
  '458e37b5-5d28-45fd-8292-4a6a844c18af',
  '44f9eac0-39b3-4b8b-b2e2-11bb57008005',
  'f337d8b5-e3bf-4ec5-bb24-31b696d14073'],
 'embeddings': None,
 'documents': ['1   +   1',
  '2   +   3',
  '6   x   5',
  'W h a t   d i d   t h e   s t o n e   s a y   t o   t h e   m o o n ?',
  'W h a t   a   g i r l   w a n t ?',
  'H o w   t o   g e t   a   l o t   o f   g o l d ?'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'output': '2', 'input': '1 + 1'},
  {'output': '5', 'input': '2 + 3'},
  {'input': '6 x 5', 'output': '30'},
  {'output': 'He is singing a song like he always need her',
   'input': 'What did the stone say to the moon?'},
  {'output': 'There is no one size fit all', 'input': 'What a girl want?'},
  {'input': 'How to get a lot of gold?',
   'output': 'type .. iseedeadpeople and press Enter'}]}

In [32]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2
)

In [33]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.format(input="What's 3+3?"))

14:04:52.098 POST 127.0.0.1/api/embed
Human: 2 + 3
AI: 5
Human: 1 + 1
AI: 2


In [34]:
vector_store.similarity_search_with_relevance_scores("What's 3+3?")

14:04:59.981 POST 127.0.0.1/api/embed


[(Document(id='65622760-a30e-4e87-a4e6-2cf24886ecbb', metadata={'input': '2 + 3', 'output': '5'}, page_content='2   +   3'),
  0.6145217203588764),
 (Document(id='478870b7-a70c-40f1-bb4a-e3cafaed92aa', metadata={'output': '2', 'input': '1 + 1'}, page_content='1   +   1'),
  0.3894007353589096),
 (Document(id='c08b682b-3aaa-4d37-bf13-5225b86711bf', metadata={'output': '30', 'input': '6 x 5'}, page_content='6   x   5'),
  0.27119939294792517),
 (Document(id='44f9eac0-39b3-4b8b-b2e2-11bb57008005', metadata={'output': 'There is no one size fit all', 'input': 'What a girl want?'}, page_content='W h a t   a   g i r l   w a n t ?'),
  0.21560508665744826)]

In [35]:
print(few_shot_prompt.format(input="What's a man want?"))

14:05:02.977 POST 127.0.0.1/api/embed
Human: What a girl want?
AI: There is no one size fit all
Human: How to get a lot of gold?
AI: type .. iseedeadpeople and press Enter


In [36]:
vector_store.similarity_search_with_relevance_scores("What's a man want?")

14:05:06.572 POST 127.0.0.1/api/embed


[(Document(id='44f9eac0-39b3-4b8b-b2e2-11bb57008005', metadata={'output': 'There is no one size fit all', 'input': 'What a girl want?'}, page_content='W h a t   a   g i r l   w a n t ?'),
  0.3662080832666378),
 (Document(id='f337d8b5-e3bf-4ec5-bb24-31b696d14073', metadata={'output': 'type .. iseedeadpeople and press Enter', 'input': 'How to get a lot of gold?'}, page_content='H o w   t o   g e t   a   l o t   o f   g o l d ?'),
  0.32241055343381464),
 (Document(id='458e37b5-5d28-45fd-8292-4a6a844c18af', metadata={'output': 'He is singing a song like he always need her', 'input': 'What did the stone say to the moon?'}, page_content='W h a t   d i d   t h e   s t o n e   s a y   t o   t h e   m o o n ?'),
  0.3172409052891654),
 (Document(id='478870b7-a70c-40f1-bb4a-e3cafaed92aa', metadata={'input': '1 + 1', 'output': '2'}, page_content='1   +   1'),
  0.20479366137113075)]